In [ ]:
import os
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import re
from scipy.stats import ks_2samp, ttest_ind, ttest_1samp

In [ ]:
def parse_sist_melt(fn):
    """Parse raw SIST melt output; return an array of probabilities"""
    data = []
    with open(fn, 'r') as f:
        for line in f:
            line = line.strip()
            if 'Position' in line or 'WARNING' in line:
                continue
            line = line.split()
            line[0], line[1], line[2], line[3] = int(line[0])-1, float(line[1]), float(line[2]), float(line[3])
            data.append(line[1])
    return np.array(data)

def parse_sist_Z(fn):
    """Parse raw SIST melt output; return an array of probabilities"""
    data = []
    with open(fn, 'r') as f:
        for line in f:
            line = line.strip()
            if 'Position' in line or 'WARNING' in line:
                continue
            line = line.split()
            line[0], line[1], line[2], line[3] = int(line[0])-1, float(line[1]), float(line[2]), float(line[3])
            data.append(line[2])
    return np.array(data)

def parse_sist_cruciform(fn):
    """Parse raw SIST melt output; return an array of probabilities"""
    data = []
    with open(fn, 'r') as f:
        for line in f:
            line = line.strip()
            if 'Position' in line or 'WARNING' in line:
                continue
            line = line.split()
            line[0], line[1], line[2], line[3] = int(line[0])-1, float(line[1]), float(line[2]), float(line[3])
            data.append(line[3])
    return np.array(data)

## Enrichment Analysis

In [ ]:
centromere = '2'
long_name = ''
cen_len = {'2': 24561, '3': 103827, '4': 93914, 'X': 70181, 'Y': 139957}
master_dict = {}
cen_master_dict = {}

for n in os.listdir("control/dna/C{}".format(centromere)):
    contig = '_'.join(n.split('-')[2].split('_')[:-1])
    start = int(n.split('-')[2].split('_')[-1])
    end =  int(n.split('-')[3])
    
    contig_identifier = '-'.join([contig, str(start)])

    fns = glob.glob('R_gquad_results/*/*{}.csv'.format(contig))
    for fn in fns:
        if contig_identifier not in master_dict:
            master_dict[contig_identifier] = np.zeros((cen_len[centromere],))
        
            
        arr = master_dict[contig_identifier].copy()

        df = pd.read_csv(fn)
        if len(df[df['sequence_position'] == '-']) >= 1:
            continue
        df = df[df['sequence_position'] >= start ]
        df = df[df['sequence_position'] <= end ]

        try:

            for s_pos, s_len, s_lik in zip(df['sequence_position'], df['sequence_length'], df['likeliness']):
                if s_lik == '*':
                    num = 1
                elif s_lik == '**':
                    num = 2
                elif s_lik == '***':
                    num = 3
                
                s_start = s_pos - start
                arr[int(s_start): (int(s_start) + int(s_len) + 1)] += num
                
        except:

            for s_pos, s_len in zip(df['sequence_position'], df['sequence_length']):
                num = 2
                
                s_start = s_pos - start
                arr[int(s_start): (int(s_start) + int(s_len) + 1)] += num
        
        master_dict[contig_identifier] = arr.copy()


# For the centromere
for n in [n.replace('\uf03a', ':') for n in os.listdir('cen/dna/C{}'.format(centromere))]:
    if '.fai' in n:
        continue
    contig = n.split('-')[1]
    
    contig_identifier = contig

    fns = glob.glob('R_gquad_results/*/*{}.csv'.format(contig))
    for fn in fns:
        if contig_identifier not in cen_master_dict:
            cen_master_dict[contig_identifier] = np.zeros((cen_len[centromere],))
        else:
            arr = cen_master_dict[contig_identifier].copy()

        
        df = pd.read_csv(fn)
        if len(df[df['sequence_position'] == '-']) >= 1:
            continue
        try:
            
            for s_pos, s_len, s_lik in zip(df['sequence_position'], df['sequence_length'], df['likeliness']):
                if s_lik == '*':
                    num = 1
                elif s_lik == '**':
                    num = 2
                elif s_lik == '***':
                    num = 3
                
                s_start = s_pos
                arr[int(s_start): (int(s_start) + int(s_len) + 1)] += num
                
        except:
            for s_pos, s_len in zip(df['sequence_position'], df['sequence_length']):
                num = 2
                
                s_start = s_pos
                arr[int(s_start): (int(s_start) + int(s_len) + 1)] += num
        
        cen_master_dict[contig_identifier] = arr.copy()


all_ks_avgs = []
cen_iden = list(cen_master_dict.keys())[0]

for key, values in master_dict.items():
    all_ks_avgs.append(ks_2samp(cen_master_dict[cen_iden], values)[1])
print('Average P Value for centromere', centromere, ":", np.mean(np.array(all_ks_avgs)))


hue = []
data = []
names = []
t_cen = []
t_con = []
minl = 0

sdata = np.mean(cen_master_dict[cen_iden])
t_cen.append(sdata)
data.append(sdata)
hue.append(1)
names.append('Cen')
np.savetxt("".join(["control/csv-results/long-centromere-", centromere, "-gquad" + long_name + ".csv"]), data, delimiter=",")

data = []
names = []

for key, values in master_dict.items():
    sdata = np.mean(values)
    t_con.append(sdata)
    data.append(sdata)
    hue.append(0)
    names.append('Control')
np.savetxt("".join(["control/csv-results/long-control-", centromere, "-gquad" + long_name + ".csv"]), data, delimiter=",")

## Ranking Contigs based on GQuad

In [ ]:
from Bio import SeqIO

# This creates a dictionary of all lengths of all contigs
sequence_lengths = {}
for seq_record in SeqIO.parse('dm6.fasta', "fasta"):
    sequence_lengths[seq_record.id] = len(seq_record)

In [ ]:
master_dict_contig = {}

for n in sequence_lengths.keys():
    fns = glob.glob('R_gquad_results/*/*{}.csv'.format(n))
    for fn in fns:
        if n not in master_dict_contig:
            master_dict_contig[n] = np.zeros((sequence_lengths[n],))
        
            
        arr = master_dict_contig[n].copy()

        df = pd.read_csv(fn)
        if len(df[df['sequence_position'] == '-']) >= 1:
            continue

        try:

            for s_pos, s_len, s_lik in zip(df['sequence_position'], df['sequence_length'], df['likeliness']):
                if s_lik == '*':
                    num = 1
                elif s_lik == '**':
                    num = 2
                elif s_lik == '***':
                    num = 3
                
                s_start = s_pos 
                arr[int(s_start): (int(s_start) + int(s_len) + 1)] += num
                
        except:

            for s_pos, s_len in zip(df['sequence_position'], df['sequence_length']):
                num = 2
                
                s_start = s_pos
                arr[int(s_start): (int(s_start) + int(s_len) + 1)] += num
        
        master_dict_contig[n] = arr.copy()
    
for key, values in master_dict_contig.items():
    master_dict_contig[key] = values.mean()

master_list_contig = []
for key, value in master_dict_contig.items():
    master_list_contig.append([key, value])


In [ ]:
import csv
  
master_list_contig.sort(key = lambda x: x[1], reverse=True)

# field names 
fields = ['Name', 'Average Likelihood'] 
    
# data rows of csv file 
rows = master_list_contig
  
with open('rank_non_b_with_gquad.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(rows)

## GQuad pie chart

In [ ]:
total_cen = None
total_control = None

cens_l = []
controls_l = []

In [ ]:
centromere = 'Y'
long_name = ''
cen_len = {'2': 24561, '3': 103827, '4': 93914, 'X': 70181, 'Y': 139957}
master_dict = {}
cen_master_dict = {}
non_b = 'str'

for n in os.listdir("control/dna/C{}".format(centromere)):
    contig = '_'.join(n.split('-')[2].split('_')[:-1])
    start = int(n.split('-')[2].split('_')[-1])
    end =  int(n.split('-')[3])
    
    contig_identifier = '-'.join([contig, str(start)])

    control_dna_dict = {
        'aphased': 0,
        'gquad': 0,
        'hdna': 0,
        'slipped': 0,
        'str': 0,
        'tfo': 0,
        'zdna': 0
    }

    cen_dna_dict = {
        'aphased': 0,
        'gquad': 0,
        'hdna': 0,
        'slipped': 0,
        'str': 0,
        'tfo': 0,
        'zdna': 0
    }

    fns = glob.glob('R_gquad_results/{}/*{}.csv'.format(non_b, contig))
    for fn in fns:

        if contig_identifier not in master_dict:
            master_dict[contig_identifier] = np.zeros((cen_len[centromere],))
        
            
        arr = master_dict[contig_identifier].copy()

        key=None

        if 'aphased_' in fn:
            key = 'aphased'
        if 'gquad_' in fn:
            key = 'gquad'
        if 'hdna_' in fn:
            key = 'hdna'
        if 'slipped_' in fn:
            key = 'slipped'
        if 'str_' in fn:
            key = 'str'
        if 'tfo_' in fn:
            key = 'tfo'
        if 'zdna_' in fn:
            key = 'zdna'

        df = pd.read_csv(fn)
        if len(df[df['sequence_position'] == '-']) >= 1:
            continue
        df = df[df['sequence_position'] >= start ]
        df = df[df['sequence_position'] <= end ]

        try:

            for s_pos, s_len, s_lik in zip(df['sequence_position'], df['sequence_length'], df['likeliness']):
                num = 1
                s_start = s_pos - start
                arr[int(s_start): (int(s_start) + int(s_len) + 1)] += num
                number_of_stars = ( num * int(s_len) )
                control_dna_dict[key] += number_of_stars
                
        except:

            for s_pos, s_len in zip(df['sequence_position'], df['sequence_length']):
                num = 1
                s_start = s_pos - start
                arr[int(s_start): (int(s_start) + int(s_len) + 1)] += num
                number_of_stars = ( num * int(s_len) )
                control_dna_dict[key] += number_of_stars
        
        master_dict[contig_identifier] = arr.copy()


# For the centromere
for n in [n.replace('\uf03a', ':') for n in os.listdir('cen/dna/C{}'.format(centromere))]:
    if '.fai' in n:
        continue
    contig = n.split('-')[1]
    
    contig_identifier = contig

    fns = glob.glob('R_gquad_results/{}/*{}.csv'.format(non_b, contig))
    for fn in fns:
        if contig_identifier not in cen_master_dict:
            cen_master_dict[contig_identifier] = np.zeros((cen_len[centromere],))
        else:
            arr = cen_master_dict[contig_identifier].copy()
        
        if 'aphased_' in fn:
            key = 'aphased'
        if 'gquad_' in fn:
            key = 'gquad'
        if 'hdna_' in fn:
            key = 'hdna'
        if 'slipped_' in fn:
            key = 'slipped'
        if 'str_' in fn:
            key = 'str'
        if 'tfo_' in fn:
            key = 'tfo'
        if 'zdna_' in fn:
            key = 'zdna'

        
        df = pd.read_csv(fn)
        if len(df[df['sequence_position'] == '-']) >= 1:
            continue
        try:
            
            for s_pos, s_len, s_lik in zip(df['sequence_position'], df['sequence_length'], df['likeliness']):
                num = 1
                s_start = s_pos
                arr[int(s_start): (int(s_start) + int(s_len) + 1)] += num
                number_of_stars = ( num * int(s_len) )
                cen_dna_dict[key] += number_of_stars
                
        except:
            for s_pos, s_len in zip(df['sequence_position'], df['sequence_length']):
                num = 1
                s_start = s_pos
                arr[int(s_start): (int(s_start) + int(s_len) + 1)] += num
                number_of_stars = ( num * int(s_len) )
                cen_dna_dict[key] += number_of_stars
        
        cen_master_dict[contig_identifier] = arr.copy()


all_ks_avgs = []
temp_controls_l = []
cen_iden = list(cen_master_dict.keys())[0]
cens_l.append(np.mean(cen_master_dict[cen_iden]))
counter = 1
for key, values in master_dict.items():
    temp_controls_l.append(np.mean(values))
    all_ks_avgs.append(ks_2samp(cen_master_dict[cen_iden], values)[1])
    if counter >= 50:
        controls_l.append(temp_controls_l)
        break 
    counter += 1


hue = []
data = []
names = []
t_cen = []
t_con = []
minl = 0

sdata = np.mean(cen_master_dict[cen_iden])
t_cen.append(sdata)
data.append(sdata)
hue.append(1)
names.append('Cen')

data = []
names = []

for key, values in master_dict.items():
    sdata = np.mean(values)
    t_con.append(sdata)
    data.append(sdata)
    hue.append(0)
    names.append('Control')

In [ ]:
ttest_1samp(np.mean(
    np.array(controls_l),
    axis=0,
).tolist(), np.mean(cens_l))

In [ ]:

if total_cen == None:
    total_cen = cen_dna_dict
else:
    for key, value in cen_dna_dict.items():
        total_cen[key] += value

if total_control == None:
    total_control = control_dna_dict
else:
    for key, value in control_dna_dict.items():
        total_control[key] += value

print('Cen', total_cen)
print('Control', total_control)

In [ ]:
# Record the outputs from above into the dictionary below and run the next cell for the percents.
# Replace the 0s with the outputs from above

# The Regular Method Totals
total_cen_1 = {'aphased': 0, 'gquad': 0, 'hdna': 0, 'slipped': 0, 'str': 0, 'tfo': 0, 'zdna': 0}
total_control_1 ={'aphased': 0, 'gquad': 0, 'hdna': 0, 'slipped': 0, 'str': 0, 'tfo': 0, 'zdna': 0}

# Different method where we addonly 1 and not the total likelihood
total_cen_2 = {'aphased': 0, 'gquad': 0, 'hdna': 0, 'slipped': 0, 'str': 0, 'tfo': 0, 'zdna': 0}
total_control_2 = {'aphased': 0, 'gquad': 0, 'hdna': 0, 'slipped': 0, 'str': 0, 'tfo': 0, 'zdna': 0}

In [ ]:
total = 0
dict_of_int = total_control_2
for key, value in dict_of_int.items():
    total += value

for key, value in dict_of_int.items():
    dict_of_int[key] = value/total

dict_of_int